### A BERTopic-ról hasznos dokumentumok
https://maartengr.github.io/BERTopic/algorithm/algorithm.html#6-optional-fine-tune-topic-representation

https://towardsdatascience.com/topics-per-class-using-bertopic-252314f2640

https://people.inf.elte.hu/csa/html/szinek.htm

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sentence_transformers import SentenceTransformer

import pickle
import huspacy


## 1. Beolvassuk az előfeldolgozott korpuszokat. Később választhatunk, hogy a lemmatizált vagy a lemmatizált és stop szavaktól megszűrt korpuszon dolgozunk-e.

In [ ]:
meta = pickle.load(open("../data/meta.pkl", "rb"))
lemmatized = pickle.load(open("../data/lemmatized.pkl", "rb"))
#pos = pickle.load(open("../data/pos.pkl", "rb"))
#tokens =  pickle.load(open("../data/tokenized.pkl", "rb"))
#doc_stop = pickle.load(open("../data/no_stopword.pkl", "rb")) ### kevesebb stopszóval
doc_stop_2 = pickle.load(open("../data/stopword_filtered.pkl", "rb")) ### több stop szóval
all_docs = pickle.load(open("../data/docs.pkl", "rb"))

## 2. Készítünk egy listát az előre megadott témák szavaiból

In [ ]:
seed_topic_list = [["szabadidő", "szabadidőtök", "szabadidőd", "szabadidődet"],
                   ["nyelv", "nyelvtanulás", "nyelvvizsga"],
                   ["sport", "sportol", "sportolás"],
                   ["ismerkedés", 'megismerkedik', "megismer"],
                   ["olvas", "olvasás", "könyv"],
                   ['külföld', 'külföldi', 'utazik'],
                   ["magyarország"],
                   ["social", "media", "facebook", "facebookon", 'instagram', 'instagramm', 'instagrammom', 'instagrammon']]
#["igen", "ja", "persze", "aha", "hum"], ["laugh", "nevet", "vicces"], ["placeholder"]

In [ ]:
## 3. Topic modellezünk. Beadjuk seednek az előre megadott témákat, így azokat könnyebben azonosítja a modell. Lekérjük minden topik leggyakoribb 40 szavát. Eldöntjük, hogy a lemmatizált vagy a stop szavazott adaton dolgozunk-e. 3 féle modellt kipróbálunk.

## 3.1 Modell: BERTopic

In [ ]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance

key = KeyBERTInspired()
mm =  MaximalMarginalRelevance(diversity=0.3)
btm_jo_seed = BERTopic("hungarian", representation_model=[key, mm], seed_topic_list=seed_topic_list, min_topic_size = 50, calculate_probabilities=True)
topics, probs = btm_jo_seed.fit_transform(doc_stop_2)

In [ ]:
btm_jo_seed_ti= btm_jo_seed.get_topic_info()
btm_jo_seed_ti

In [ ]:
import pandas as pd
probs_df=pd.DataFrame(probs)
probs_df['main percentage'] = pd.DataFrame({'max': probs_df.max(axis=1)})
probs_df

## Reduce outliers. Kétfélr módszert próbálunk ki: valószínűség és eloszlás alapján.

## Valószínűség alapján

In [ ]:
new_topics = btm_jo_seed.reduce_outliers(doc_stop_2, topics, probabilities=probs, strategy="probabilities")

## Frissítjük a modellt az outlierek kizárása után létrehozott új topikokkal és topikeloszlásokkal

In [ ]:
import pandas as pd
btm_jo_seed.update_topics(doc_stop_2, new_topics)
documents = pd.DataFrame({"Document": doc_stop_2, "Topic": new_topics})
btm_jo_seed._update_topic_size(documents)

In [ ]:
btm_jo_seed_probs_ti = btm_jo_seed.get_topic_info()
btm_jo_seed_probs_ti

In [ ]:
import pandas as pd
probs_df_2=pd.DataFrame(probs)
probs_df_2['main percentage'] = pd.DataFrame({'max': probs_df.max(axis=1)})
probs_df_2

## Eloszlás alapján (default).

In [ ]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance

key = KeyBERTInspired()
mm =  MaximalMarginalRelevance(diversity=0.3)
sima_btm = BERTopic("hungarian", representation_model=[key, mm], seed_topic_list=seed_topic_list, min_topic_size = 50, calculate_probabilities=True)
topics, probs = sima_btm.fit_transform(doc_stop_2)

In [ ]:
new_topics_2 = sima_btm.reduce_outliers(doc_stop, topics)

In [ ]:
import pandas as pd
sima_btm.update_topics(doc_stop, new_topics_2)
documents_2 = pd.DataFrame({"Document": doc_stop, "Topic": new_topics_2})
sima_btm._update_topic_size(documents_2)

In [ ]:
btm_sima_ti_new = sima_btm.get_topic_info()
btm_sima_ti_new

## 3.2 Modell: Sentence Transformer

In [ ]:
sentence_model = SentenceTransformer("NYTK/sentence-transformers-experimental-hubert-hungarian")
sentence_transformer_lemmatized = BERTopic(embedding_model=sentence_model, min_topic_size = 30, seed_topic_list=seed_topic_list)
topics, probs = sentence_transformer_lemmatized.fit_transform(lemmatized)

In [ ]:
sentence_topic_info_lemmatized = sentence_transformer_lemmatized.get_topic_info()
sentence_topic_info_lemmatized

## 3.3 Modell: Huspacy

In [ ]:
nlp = huspacy.load()
spacy_lemmatized = BERTopic(embedding_model=nlp, min_topic_size = 30, seed_topic_list=seed_topic_list)
topics, probs = spacy_lemmatized.fit_transform(lemmatized)

In [ ]:
spacy_topic_info_lemmatized = spacy_lemmatized.get_topic_info()
spacy_topic_info_lemmatized

## 3.4 HDBscan

In [ ]:
hdbscan_model = HDBSCAN(min_cluster_size=30, metric='euclidean', cluster_selection_method='eom', prediction_data=True, min_samples=10)
hdbscan_lemmatized = BERTopic(hdbscan_model=hdbscan_model, seed_topic_list=seed_topic_list)
topics, probs = hdbscan_lemmatized.fit_transform(lemmatized)

In [ ]:
hdbscan_lemmatized_topic_info = hdbscan_lemmatized.get_topic_info()
hdbscan_lemmatized_topic_info

## 4. Lementjük a modelleket és a topikokat

In [ ]:
def save_model(model_name, model_path):
    model_name.save(model_path, serialization="pickle")

In [ ]:
a = save_model(btm_jo_seed,"../data/models/bert_stop_2_probabilities")

In [ ]:
def save_topic_info(model_topic_info, topic_path):
    model_topic_info.to_csv(topic_path, sep=",", index=False)

In [ ]:
b = save_topic_info(btm_jo_seed_probs_ti, "../data/topic_info/bert_stop_2_probabilities.csv")

## 5. Összevonjuk a hasonló topikokat

## Először betöltjük a használni kívánt modellt

In [ ]:
loaded_model_path = "../data/models/bert_stop_2_probabilities"
loaded_model = BERTopic.load(loaded_model_path)
print("A modellt betöltöttük")

In [ ]:
loaded_model.get_topic_info()

## 5.1 Lekérjük a similarity hetmapet, hogy vizuálisan lássuk, mely témák hasonlóak

In [ ]:
loaded_model.visualize_heatmap()

## 5.2 Lekérjük a témák hierarchiáját, hogy lássuk a témák alá-fölé rendeltségének összefüggéseit

In [ ]:
hierarchical_topics = loaded_model.hierarchical_topics(doc_stop_2)

# Visualize these representations
loaded_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

## 5.2 Megnézzük, mely témák hasonlítanak számszerűsítve

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

distance_matrix = cosine_similarity(np.array(loaded_model.topic_embeddings_))
dist_df = pd.DataFrame(distance_matrix, columns=loaded_model.topic_labels_.values(),
                       index=loaded_model.topic_labels_.values())

tmp = []
for rec in dist_df.reset_index().to_dict('records'):
    t1 = rec['index']
    for t2 in rec:
        if t2 == 'index':
            continue
        tmp.append(
            {
                'topic1': t1,
                'topic2': t2,
                'distance': rec[t2]
            }
        )

pair_dist_df = pd.DataFrame(tmp)

pair_dist_df = pair_dist_df[(pair_dist_df.topic1.map(
      lambda x: not x.startswith('-1'))) &
            (pair_dist_df.topic2.map(lambda x: not x.startswith('-1')))]
pair_dist_df = pair_dist_df[pair_dist_df.topic1 < pair_dist_df.topic2]
pair_dist_df.sort_values('distance', ascending = False).head(20)

## 5.3 Megnézzük, mely témák hasonlítanak nagyobb, mint 85%-ban. Lementjük az adatot.

In [ ]:
similar_topics = pair_dist_df[pair_dist_df["distance"] > 0.85]
similar_topics

In [ ]:
similar_topics.to_csv("../data/similar_topics_85.csv")

## 5.6 Áttanulmányozzuk a lementett csv-t és eldöntjük, mely témákat érdemes összevonni a számszerűsített hasonlóság miatt. Megadjuk az összevonandó témákat, majd összevonjuk ezeket.

In [ ]:
topics_to_merge = [[0, 7, 8, 13, 15, 16, 20, 26, 27, 30, 35, 36, 38, 41, 42], [2, 38, 20], [5, 39], [1,6], [18, 19]] #1.csoport: diskurzus előremozdítása, többnyire:igen, aha, hüm, ja 2. csoport: hümögés, 3. csoport: placeholder = stopszavak, 4. csoport: nevetés, 5. csoport: nyelvek
loaded_model.merge_topics(doc_stop_2, topics_to_merge)

In [ ]:
merged_ti = loaded_model.get_topic_info()
merged_ti

## 5.7 Lementjük a frissített, összevont témákat tartalmazó topic modellt és a topikokat

In [ ]:
st = save_topic_info(merged_ti, "../data/topic_info/bert_stop_2_probabilities_merged_85.csv")

In [ ]:
sm = save_model(loaded_model,"../data/models/bert_stop_2_probabilities_merged_85")